In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
import json
import shutil
import matplotlib.pyplot as plt

In [ ]:
from kaggle_secrets import UserSecretsClient
import wandb

user_secrets = UserSecretsClient()
wandb_key = user_secrets.get_secret("wandb-key")

wandb.login(key=wandb_key)

In [ ]:
shutil.rmtree('./data', ignore_errors=True)
Path("./data/train/images").mkdir(parents=True, exist_ok=True)
Path("./data/train/labels").mkdir(parents=True, exist_ok=True)
Path("./data/val/images").mkdir(parents=True, exist_ok=True)
Path("./data/val/labels").mkdir(parents=True, exist_ok=True)

In [ ]:
sample_subm = pd.read_csv('../input/detection-of-human-silhouettes-in-forest-images/omsk/sample_solution.csv')
train = pd.read_csv('../input/detection-of-human-silhouettes-in-forest-images/train_dataset_train/train.csv')
test_dir = '../input/detection-of-human-silhouettes-in-forest-images/test_dataset_test/test'
train_dir = '../input/detection-of-human-silhouettes-in-forest-images/train_dataset_train/train'
train_img_dir = './data/train/images'
train_labels_dir = './data/train/labels'
val_img_dir = './data/val/images'
val_labels_dir = './data/val/labels'

sample_subm['region_shape'] = sample_subm['region_shape'].astype('object')

Train / val split

In [ ]:
# OLD
val_part = pd.concat([train[train.count_region != 0].sample(frac=0.2, random_state=42), train[train.count_region == 0].sample(frac=0.01, random_state=42)])

# train_part = train[~train.isin(val_part)].dropna() # for using all images from train
train_part = train

train_part = pd.concat([train_part[train_part.count_region != 0], train_part[train_part.count_region == 0].sample(frac=0.03, random_state=42)])

In [ ]:
# val_part = pd.concat([train[train.count_region != 0].sample(frac=0.2, random_state=42), train[train.count_region == 0].sample(frac=0.001, random_state=42)])
# train_part = train[~train.isin(val_part)].dropna()
# train_part = pd.concat([train_part[train_part.count_region != 0], train_part[train_part.count_region == 0].sample(frac=0.003, random_state=42)])

In [ ]:
# all to train:
# val_part = pd.concat([train[train.count_region != 0].sample(frac=0.3, random_state=42), train[train.count_region == 0].sample(frac=0.001, random_state=42)])
# train_part = pd.concat([train_part[train_part.count_region != 0], train_part[train_part.count_region == 0].sample(frac=0.003, random_state=42)])

In [ ]:
len(train_part), len(train_part[train_part.count_region > 0]), len(val_part), len(val_part[val_part.count_region > 0])

In [ ]:
def yolobbox2bbox(x,y,w,h):
    x1, y1 = x-w/2, y-h/2
    x2, y2 = x+w/2, y+h/2
    return x1, y1, x2, y2

# Convert Coco bb to Yolo
def coco_to_yolo(x1, y1, w, h, image_w, image_h):
    return [((2*x1 + w)/(2*image_w)) , ((2*y1 + h)/(2*image_h)), w/image_w, h/image_h]

Example

In [ ]:
img = cv2.imread(f'{train_dir}/4269.JPG')
h, w = img.shape[0], img.shape[1]

coords_str = train[train.count_region != 0].iloc[6]['region_shape'].replace("'", "")
circle_coords = json.loads(coords_str)
yolo_coords_f = ''
for item in circle_coords:
    cx = item['cx']
    cy = item['cy']
    r = int(item['r'] // 1.5)
#     r = item['r']
    top_left_cornel = (cx - r, cy - r)
    bottom_right_cornel = (cx + r, cy + r)

    yolo_coords = coco_to_yolo(*top_left_cornel, bottom_right_cornel[0] - top_left_cornel[0], bottom_right_cornel[1] - top_left_cornel[1], w, h)
    x1, y1 = top_left_cornel[0], top_left_cornel[1]
    w, h = bottom_right_cornel[0] - top_left_cornel[0], bottom_right_cornel[1] - top_left_cornel[1]
    img = cv2.rectangle(img, (x1, y1), (x1+w, y1+h), (255,0,0), 2)
    
    label = '0 ' + ' '.join(list(map(str, yolo_coords))) + '\n'
    yolo_coords_f = yolo_coords_f + label
plt.figure(figsize=(30, 20))
plt.imshow(img)
print(yolo_coords_f)

In [ ]:
def create_lables(df ,train_img_dir, train_labels_dir):
    print('imgs with labels: \n')
    for i, row in df.iterrows():
        shutil.copy(f'{train_dir}/{row["ID_img"]}', train_img_dir)
        if row['count_region'] == 0:
            open(f'{train_labels_dir}/{row["ID_img"].split(".")[0]}.txt', 'a').close()
        else:
            print(f'{train_img_dir}/{row["ID_img"]}')
            img = cv2.imread(f'{train_img_dir}/{row["ID_img"]}')
            h, w = img.shape[0], img.shape[1]

            coords_str = row['region_shape'].replace("'", "")
            circle_coords = json.loads(coords_str)
            yolo_coords_f = ''

            for item in circle_coords:
                cx = item['cx']
                cy = item['cy']
                r = item['r']
                top_left_cornel = (cx - r, cy - r)
                bottom_right_cornel = (cx + r, cy + r)

                yolo_coords = coco_to_yolo(*top_left_cornel, bottom_right_cornel[0] - top_left_cornel[0], bottom_right_cornel[1] - top_left_cornel[1], w, h)

                label = '0 ' + ' '.join(list(map(str, yolo_coords))) + '\n'
                yolo_coords_f = yolo_coords_f + label
            with open(f'{train_labels_dir}/{row["ID_img"].split(".")[0]}.txt', 'a') as label_txt:
                label_txt.write(yolo_coords_f)

In [ ]:
create_lables(train_part, train_img_dir, train_labels_dir)
create_lables(val_part, val_img_dir, val_labels_dir)

In [ ]:
category_names = ['human']
category_names = {k: v for v, k in enumerate(category_names)}
names_str = " \n ".join([f'{item[1]}: {item[0]}' for item in list(zip(category_names.keys(), category_names.values()))])
yaml_content = f"""
train: /kaggle/working/data/train/images
val: /kaggle/working/data/val/images

# number of classes
nc: 1

# class names
names: \n {names_str}
"""

with open('./data/dataset.yaml', 'w') as f:
    f.write(yaml_content)

In [ ]:
!cat ./data/dataset.yaml

YOLO

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r ./yolov5/requirements.txt

train

In [ ]:
# YOLOv5l YOLOv5x YOLOv5s6 YOLOv5m6 YOLOv5l6 YOLOv5x6

yolov5l

In [ ]:
# !cd yolov5 && python train.py --img 1280 --batch 16 --epochs 30 --data ../data/dataset.yaml --weights yolov5s.pt
# !cd yolov5 && python -m torch.distributed.run --nproc_per_node 2 train.py --img 1920 --batch 4 --epochs 90 --data ../data/dataset.yaml --weights yolov5l.pt --hyp hyp.scratch-med.yaml --device 0,1 --cache disk

yolov5s / m

In [ ]:
# !cd yolov5 && python train.py --img 1920 --batch 16 --epochs 20 --data ../data/dataset.yaml --weights yolov5s.pt --hyp hyp.scratch-med.yaml

In [ ]:
!cd yolov5 && python -m torch.distributed.run --nproc_per_node 2 train.py --img 1920 --batch 6 --epochs 90 --data ../data/dataset.yaml --weights yolov5m.pt --hyp hyp.scratch-med.yaml --device 0,1 --cache disk

Inference

In [ ]:
# --iou 0.05 --agnostic --conf 0.15

In [ ]:
# !ls ./yolov5/runs/train/
# [x[0] for x in os.walk('./yolov5/runs/train/')]
# sorted(next(os.walk('./yolov5/runs/train/'))[1])

In [ ]:
exp_last = sorted(next(os.walk('./yolov5/runs/train/'))[1])[-1]
!echo $exp_last

Download weights

<a href="./yolov5/runs/train/exp7/weights/last.pt"> Download File </a>

In [ ]:
from IPython.display import FileLink
FileLink(f'./yolov5/runs/train/{exp_last}/weights/best.pt')

In [ ]:
!cd yolov5 && python detect.py --augment --img 1920 --conf 0.25 --iou 0.45 --source '../../input/detection-of-human-silhouettes-in-forest-images/test_dataset_test/test' --weights runs/train/$exp_last/weights/best.pt --save-txt --save-conf --half

In [ ]:
exp_detect_last = sorted(next(os.walk('./yolov5/runs/detect/'))[1])[-1]

In [ ]:
# load weights
# !cd yolov5 && python detect.py --augment --img 1920 --conf 0.2 --source '../../input/detection-of-human-silhouettes-in-forest-images/test_dataset_test/test' --weights ../../input/detection-of-human-silhouettes-in-forest-weights/best.pt --save-txt --save-conf --half

In [ ]:
def get_solution_labels_df(path_to_txt_folder):
    simple_solution = []
    for detection_file in os.listdir(path_to_txt_folder):
        img_name = sample_subm[sample_subm['ID_img'].str.contains(detection_file.split('.')[0])]['ID_img'].iloc[0]
        with open(path_to_txt_folder + detection_file, 'r') as f:
            data = f.read()
            data = [i for i in data.split('\n') if i != '']
        for line in data:
            val = [float(i) for i in line.split()]
            cls, xywh, conf = val[0], val[1:5], val[5]
            center_x, center_y, width, height = xywh
            xmin = center_x - (width / 2)
            xmax = center_x + (width / 2)
            ymin = center_y - (height / 2)
            ymax = center_y + (height / 2)
            simple_solution.append([img_name, cls, conf, xmin, xmax, ymin, ymax, center_x, center_y, width, height])
    return simple_solution

In [ ]:
test_predicts = get_solution_labels_df(f'./yolov5/runs/detect/{exp_detect_last}/labels/')
test_predicts = pd.DataFrame(test_predicts, columns=['ID_image', 'label', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax', 'center_x', 'center_y', 'width', 'height'])

In [ ]:
test_predicts = test_predicts[test_predicts.Conf > 0.46] # 
test_predicts.head(5)

In [ ]:
len(test_predicts), len(test_predicts['ID_image'].unique())

In [ ]:
for i, row in test_predicts.iterrows():
    img = cv2.imread(f'{test_dir}/{row["ID_image"]}')
    h, w = img.shape[0], img.shape[1]
    center_x = row['center_x'] * w
    center_y = row['center_y'] * h
    width = row['width'] * w
    height = row['height'] * h
    r = width / 2
    
    test_predicts.at[i,'cx'] = int(center_x)
    test_predicts.at[i,'cy'] = int(center_y)
    test_predicts.at[i,'r'] = int(r)
    
test_predicts['cx'] = test_predicts['cx'].astype(int)
test_predicts['cy'] = test_predicts['cy'].astype(int)
test_predicts['r'] = test_predicts['r'].astype(int)

In [ ]:
len(test_predicts), len(test_predicts['ID_image'].unique())

In [ ]:
detect_img_path = f'./yolov5/runs/detect/{exp_detect_last}'
uniq_imgs_list = test_predicts['ID_image'].unique()
img_path = f'{detect_img_path}/{uniq_imgs_list[5]}'
img = cv2.imread(img_path)
plt.figure(figsize=(30, 20))
plt.imshow(img)

In [ ]:
for i, row in sample_subm.iterrows():
    if len(test_predicts[test_predicts.ID_image == row["ID_img"]]) > 0:
        circle_coords = [f'{{"cx":{item[1]["cx"]}, "cy":{item[1]["cy"]}, "r":{int(item[1]["r"])}}}' for item in test_predicts[test_predicts.ID_image == row['ID_img']].sort_values(by=['cx', 'cy']).iterrows()]
        sample_subm.at[i, 'region_shape'] = circle_coords
    else:
        sample_subm.at[i, 'region_shape'] = 0

In [ ]:
sample_subm[sample_subm.region_shape != 0].head(20)

In [ ]:
index = 503
img = cv2.imread(f'{test_dir}/{sample_subm.iloc[index]["ID_img"]}')

coords_str = str(sample_subm.iloc[index]['region_shape']).replace("'", "")
circle_coords = json.loads(coords_str)
yolo_coords_f = ''
for item in circle_coords:
    cx = item['cx']
    cy = item['cy']
    r = int(item['r'] // 2) # old: 1.5
#     r = item['r']

    img = cv2.circle(img, (cx, cy), r, (255,0,0), 2)
    
plt.figure(figsize=(30, 20))
plt.imshow(img)

In [ ]:
# 0.6 * recall + 0.4* v_norm
# v_norm = 1 - sum((x-x_pred)^2 + (y-y_pred)^2 + (r-r_pred)^2) / const
# recall = tp /(tp+fn)

In [ ]:
sample_subm.to_csv('./baseline_0_46_m_1920_90.csv', index=False)

In [90]:
!ls yolov5

CONTRIBUTING.md  classify    models	       train.py        yolov5m.pt
LICENSE		 data	     requirements.txt  tutorial.ipynb
README.md	 detect.py   runs	       utils
__pycache__	 export.py   segment	       val.py
benchmarks.py	 hubconf.py  setup.cfg	       wandb
